In [1]:
from pyecharts.charts import Scatter, Bar, Grid
from pyecharts import options as opts
import pandas as pd

# 加载数据
file_path = './数据/合并后的数据.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# Step 1: 计算每个知识点的平均得分和答题量
knowledge_score_data = data.groupby('knowledge').agg(
    avg_score=('score_x', 'mean'),
    count=('score_x', 'size')
).reset_index()

# 将数据列强制转换为列表格式
knowledge_list = knowledge_score_data['knowledge'].tolist()
avg_score_list = knowledge_score_data['avg_score'].tolist()
count_list = knowledge_score_data['count'].tolist()

# Step 2: 创建散点图展示知识掌握水平与平均得分的关系
scatter = (
    Scatter()
    .add_xaxis(knowledge_list)
    .add_yaxis(
        "Average Score",
        avg_score_list,
        symbol_size=[min(50, max(10, c / 2)) for c in count_list],  # 控制气泡大小
        itemstyle_opts=opts.ItemStyleOpts(opacity=0.7)  # 设置透明度
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="知识掌握水平与平均得分的关系"),
        xaxis_opts=opts.AxisOpts(name="知识掌握水平", axislabel_opts={"rotate": 45}),
        yaxis_opts=opts.AxisOpts(name="平均得分", min_=0),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
    )
)

# Step 3: 创建直方图展示不同得分分布情况
score_distribution = data['score_x'].value_counts().sort_index()
score_x = score_distribution.index.tolist()
score_y = score_distribution.values.tolist()

bar = (
    Bar()
    .add_xaxis(score_x)
    .add_yaxis("答题次数", score_y)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各得分的答题次数分布"),
        xaxis_opts=opts.AxisOpts(name="得分"),
        yaxis_opts=opts.AxisOpts(name="次数"),
    )
)

# Step 4: 使用 Grid 组合展示
grid = (
    Grid(init_opts=opts.InitOpts(width="1200px", height="800px"))
    .add(scatter, grid_opts=opts.GridOpts(pos_bottom="60%", pos_left="10%"))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_left="10%"))
)

# 保存成 HTML 文件
grid.render("analysis_output.html")

'D:\\Python\\数据可视化\\实验六\\analysis_output.html'

In [2]:
from pyecharts.charts import Bar, Page, Tab
from pyecharts import options as opts
import pandas as pd

# 加载数据
file_path = './数据/合并后的数据.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# 获取所有题目 ID 列表
titles = data['title_ID'].unique()

# 创建一个 Tab 容器
tab = Tab()

# 遍历每个题目 ID，生成其得分分布直方图，并将其加入 Tab 中
for title in titles:
    # 筛选出当前题目的数据
    title_data = data[data['title_ID'] == title]
    
    # 统计当前题目的得分分布
    score_distribution = title_data['score_x'].value_counts().sort_index()
    score_x = score_distribution.index.tolist()
    score_y = score_distribution.values.tolist()
    
    # 创建直方图
    bar = (
        Bar()
        .add_xaxis(score_x)
        .add_yaxis("答题次数", score_y)
        .set_global_opts(
            title_opts=opts.TitleOpts(title=f"题目 {title} 的得分分布"),
            xaxis_opts=opts.AxisOpts(name="得分"),
            yaxis_opts=opts.AxisOpts(name="次数"),
        )
    )
    
    # 将每个题目的图表添加到 Tab 容器中，并以题目 ID 作为 Tab 名称
    tab.add(bar, f"题目 {title}")

# 保存成 HTML 文件，包含所有题目的得分分布图表
tab.render("per_question_tab.html")


'D:\\Python\\数据可视化\\实验六\\per_question_tab.html'

In [14]:
# 导入必要的库
import pandas as pd
from pyecharts.charts import Bar, Line, Pie, Scatter, Page
from pyecharts import options as opts

# 加载数据
file_path = './数据/合并后的数据.xlsx'
data = pd.read_excel(file_path)

# 确保 timeconsume 列为数值，无法转换的设置为 NaN
data['timeconsume'] = pd.to_numeric(data['timeconsume'], errors='coerce')

# 1. 编程语言使用偏好分析
language_distribution = data['method'].value_counts().reset_index()
language_distribution.columns = ['编程语言', '答题次数']

pie_language = (
    Pie()
    .add(
        "编程语言",
        [list(z) for z in zip(language_distribution['编程语言'], language_distribution['答题次数'])]
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言使用偏好"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%")
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}次"))
)

# 2. 编程语言与平均得分分析
language_score = data.groupby('method')['score_x'].mean().reset_index()
language_score.columns = ['编程语言', '平均得分']

bar_language_score = (
    Bar()
    .add_xaxis(language_score['编程语言'].tolist())
    .add_yaxis("平均得分", language_score['平均得分'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言与平均得分"),
        xaxis_opts=opts.AxisOpts(name="编程语言"),
        yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4)
    )
)

# 3. 编程语言与答题耗时分析
# 过滤掉 NaN 值
language_timeconsume = data.dropna(subset=['timeconsume']).groupby('method')['timeconsume'].mean().reset_index()
language_timeconsume.columns = ['编程语言', '平均答题耗时']

line_language_timeconsume = (
    Line()
    .add_xaxis(language_timeconsume['编程语言'].tolist())
    .add_yaxis("平均耗时 (秒)", language_timeconsume['平均答题耗时'].tolist(), label_opts=opts.LabelOpts(formatter="{c} s"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言与答题耗时"),
        xaxis_opts=opts.AxisOpts(name="编程语言"),
        yaxis_opts=opts.AxisOpts(name="平均耗时 (秒)")
    )
)

# 生成HTML看板
page = Page(layout=Page.DraggablePageLayout)
page.add(pie_language, bar_language_score, line_language_timeconsume)

# 渲染为 HTML 文件
page.render("learn.html")


'D:\\Python\\数据可视化\\实验六\\learn.html'

In [3]:
# 导入必要的库
import pandas as pd
import glob
import os

# 加载题目和学生的属性数据，只需加载一次
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

# 获取所有班级数据文件的路径
file_paths = [
    './数据/Data_SubmitRecord/SubmitRecord-Class1.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class2.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class3.csv',
    # 添加其他班级数据文件路径
]

# 遍历每个班级的数据文件，合并并保存为 Excel 文件
for file_path in file_paths:
    # 读取班级提交记录数据
    submit_records = pd.read_csv(file_path)
    
    # 提取班级名称（假设文件名包含班级信息）
    class_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # 合并数据
    merged_data = pd.merge(submit_records, question_attributes, on='title_ID', how='left')
    merged_data = pd.merge(merged_data, student_attributes, on='student_ID', how='left')
    
    # 保存合并后的数据到 Excel 文件
    output_path = f'./数据/{class_name}_merged_data.xlsx'
    merged_data.to_excel(output_path, index=False)
    print(f"合并后的数据已保存为 '{output_path}' 文件。")

合并后的数据已保存为 './数据/SubmitRecord-Class1_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class2_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class3_merged_data.xlsx' 文件。


In [5]:
import pandas as pd
from pyecharts.charts import Bar, Line, Pie, Scatter, Page
from pyecharts import options as opts
import os

# 设置数据路径，假设所有班级的 CSV 数据文件存放在 './数据/Data_SubmitRecord/' 目录下
file_paths = [
    './数据/SubmitRecord-Class1_merged_data.xlsx',
    './数据/SubmitRecord-Class2_merged_data.xlsx',
    './数据/SubmitRecord-Class3_merged_data.xlsx',
    # 添加其他班级数据文件路径
]

# 合并和清洗每个班级的数据，并分别保存为 Excel 和 HTML 看板
for file_path in file_paths:
    # 读取班级数据
    class_data = pd.read_csv(file_path)
    
    # 提取班级名称（假设文件名包含班级信息）
    class_name = os.path.splitext(os.path.basename(file_path))[0]
    class_data['class_name'] = class_name  # 添加班级列

    # 检查是否包含所需的列
    required_columns = ['method', 'score_x', 'timeconsume', 'time', 'knowledge']
    missing_columns = [col for col in required_columns if col not in class_data.columns]
    
    if missing_columns:
        print(f"{class_name} 缺少列: {missing_columns}")
        continue  # 如果缺少列，跳过该文件的处理

    # 数据清洗
    class_data['timeconsume'] = pd.to_numeric(class_data['timeconsume'], errors='coerce')
    class_data['time'] = pd.to_datetime(class_data['time'], unit='s')
    
    # 保存每个班级的清洗合并数据为 Excel 文件
    class_data.to_excel(f'./数据/{class_name}_merged_data.xlsx', index=False)

    # 1. 编程语言使用偏好分析
    language_distribution = class_data['method'].value_counts().reset_index()
    language_distribution.columns = ['编程语言', '答题次数']
    pie_language = (
        Pie()
        .add(
            "编程语言",
            [list(z) for z in zip(language_distribution['编程语言'], language_distribution['答题次数'])]
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="编程语言使用偏好"),
            legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%")
        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}次"))
    )

    # 2. 编程语言与平均得分分析
    language_score = class_data.groupby('method')['score_x'].mean().reset_index()
    language_score.columns = ['编程语言', '平均得分']
    bar_language_score = (
        Bar()
        .add_xaxis(language_score['编程语言'].tolist())
        .add_yaxis("平均得分", language_score['平均得分'].tolist())
        .set_global_opts(
            title_opts=opts.TitleOpts(title="编程语言与平均得分"),
            xaxis_opts=opts.AxisOpts(name="编程语言"),
            yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4)
        )
    )

    # 3. 编程语言与答题耗时分析
    language_timeconsume = class_data.dropna(subset=['timeconsume']).groupby('method')['timeconsume'].mean().reset_index()
    language_timeconsume.columns = ['编程语言', '平均答题耗时']
    line_language_timeconsume = (
        Line()
        .add_xaxis(language_timeconsume['编程语言'].tolist())
        .add_yaxis("平均耗时 (秒)", language_timeconsume['平均答题耗时'].tolist(), label_opts=opts.LabelOpts(formatter="{c} s"))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="编程语言与答题耗时"),
            xaxis_opts=opts.AxisOpts(name="编程语言"),
            yaxis_opts=opts.AxisOpts(name="平均耗时 (秒)")
        )
    )

    # 4. 知识点掌握情况分析
    knowledge_mastery = class_data.groupby('knowledge')['score_x'].mean().reset_index()
    knowledge_mastery.columns = ['知识点', '平均得分']
    bar_knowledge = (
        Bar()
        .add_xaxis(knowledge_mastery['知识点'].tolist())
        .add_yaxis("平均得分", knowledge_mastery['平均得分'].tolist())
        .set_global_opts(
            title_opts=opts.TitleOpts(title="知识点掌握情况"),
            xaxis_opts=opts.AxisOpts(name="知识点"),
            yaxis_opts=opts.AxisOpts(name="平均得分")
        )
    )

    # 创建每个班级的页面
    page = Page(layout=Page.DraggablePageLayout)
    page.add(pie_language, bar_language_score, line_language_timeconsume, bar_knowledge)
    
    # 渲染当前班级的页面
    page_file_path = f"./数据/{class_name}_dashboard.html"
    page.render(page_file_path)

# 生成包含所有班级链接的主页面
with open("all_classes_dashboard.html", "w", encoding="utf-8") as f:
    f.write("<html><head><title>所有班级数据看板</title></head><body>")
    f.write("<h1>所有班级数据看板</h1><ul>")
    for file_path in file_paths:
        class_name = os.path.splitext(os.path.basename(file_path))[0]
        f.write(f'<li><a href="./数据/{class_name}_dashboard.html">{class_name} 的数据看板</a></li>')
    f.write("</ul></body></html>")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 16: invalid start byte